## I analyzed the data and managed to extract a meaningful insight for this e-commerce store.

### Check out the insights at the bottom of this notebook :)

## Importing Libraries and Dataset

In [ ]:
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd 
df = pd.read_csv('/kaggle/input/e-commerce-customer-behavior-and-sales-analysis-tr/ecommerce_customer_behavior_dataset.csv')

## Analyzing the data

In [ ]:
df.head()

In [ ]:
### Checking null and data type

print(f'> Checking Null values and Data Types <'), print(f'-' * 60), df.info()
print('\n')
print(f'> Checking Null again <'), print(f'-' * 30), df.isnull().sum()


In [ ]:
## Checking the general statistics 

df.describe()

# 1. What product contributes the most for the anual revenue?

In [ ]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce') #Assure that 'Date' camp is a date
df_filter_1year = df.loc[(df['Date'] >= '2023-01-01') & (df['Date'] <= '2024-01-01')] 
df_values = df_filter_1year.groupby('Product_Category')['Total_Amount'].sum().sort_values()

plt.figure(figsize=(8, 8))
plt.pie(x = df_values, labels = df_values.index, textprops = {'fontsize':9}, autopct = '%1.1f%%', pctdistance=0.9)
plt.title('Most popular product category', size= 14)
plt.show()

## 2. Sales during the year

In [ ]:
grouping_months = df['Date'].dt.month #That returns the same index with 5000 rows, but separeted by dates
sales_month = df.groupby(grouping_months)['Total_Amount'].sum() #grouping and summing the total_amount sold in that month
#Above: It works because the "grouping_months" returns a big list, but is grouped by 'df.groupby'

plt.figure(figsize=(10,4))

plt.plot(sales_month.index, sales_month, marker='o', markevery=1, markersize=4) #Using line graphs to describe better the progression
sns.set_style('whitegrid')

plt.title(f"Sales during the year of 2023") #

# plt.yticks(sales_month, [(f'{i:,.2f}') for i in sales_month]) ### FAILED. I tried to format the numbers, but they overlapped

plt.xticks(sales_month.index, ['Jan','Feb','Mar','April','May','June','July','Aug','Sept','Oct','Nov','Dec']) 

#Above: Replacing the argument X (which is an int between 1-12) by a List with the names of the months for better visualization

plt.xticks(rotation=45)
plt.show()



### There's a big increase in sales in the first two months...

## 3. What is the most sold category in those two months?

In [ ]:
sns.set_style('whitegrid')

filtro_mes = df['Date'].loc[((df['Date'].dt.month == 1) & (df['Date'].dt.year == 2023)) | \
((df['Date'].dt.month == 2) & (df['Date'].dt.year == 2023))]
#ABOVE:

most_sold_jan_fev = df.groupby([filtro_mes.dt.month,'Product_Category'])['Total_Amount'].sum().nlargest(2).sort_values()
most_sold_jan_fev = most_sold_jan_fev.reset_index()
#ABOVE: 

#COSMETICS:
plt.figure(figsize=(8,5))
plt.bar(most_sold_jan_fev['Date'], most_sold_jan_fev['Total_Amount'], label='Electronics')
plt.title('Most sold Product in January/February')
plt.xticks(most_sold_jan_fev['Date'], ['Janeiro','Fevereiro'])

plt.yticks(
    most_sold_jan_fev['Total_Amount'],                               
    [f"${i:,.2f}" for i in most_sold_jan_fev['Total_Amount']] #LIST COMPREHENSION to format the numbers        
)

plt.legend() #TO EXHIBIT the legend "Electronics" 
plt.show()




### Electronics outperformed other products, but this big volume are maintained during the year?

In [ ]:
#Let's remove the first two months of the year and check the volume of sales in Electronics compared to other products
#Does it still outperform other product categories?

#df['Date'].info() #Checking if date is in the correct format

df['Month'] = df['Date'].dt.month #Extract month from the date column

filtering_months = df.loc[((df['Month'] >=3) & (df['Month'] <=12))] #Filtering months withour jan/fev to compare

#filtering_months['Month'].unique() #Checking if the filter works

filtering_months = filtering_months.groupby('Product_Category')['Total_Amount'].sum() 

plt.figure(figsize=(8,8))
plt.pie(filtering_months, labels=filtering_months.index, textprops={'fontsize':9}, autopct= '%1.1f%%', pctdistance=0.9)
plt.show()

### Even after filtering the outliers (jan/fev) Electronics performs way above the other products

## More explorations

In [ ]:
## In case that a website is badly designed for mobile devices, maybe this data can contribute for noticing that

print(f'The device type affects the overall experience of the user?')
print(f'-'*60)
print(f'Days/Rating')


df.loc[:10, ['Device_Type', 'Customer_Rating']] 
device_vs_d_time = df.groupby('Device_Type')['Customer_Rating'].mean()

pd.Series([(f' {i:,.2f}') for i in device_vs_d_time], device_vs_d_time.index) #Formatting the float number

In [ ]:
print(f'\nThe delivery time affects the overall experience of the user?')
print(f'-' * 60)
print(f'Days/Rating')

delivery_time_filter = df.loc[(df['Delivery_Time_Days'].isin([1,2,3,4,5,6,7,8]))]
rating_vs_d_time = df.groupby(delivery_time_filter['Delivery_Time_Days'])['Customer_Rating'].mean()
pd.Series([(f' {i:,.2f}') for i in rating_vs_d_time], index=[1,2,3,4,5,6,7,8])#Formatting the float number
#Above: Filtering 1-8 days to check if the rating increase between people who received the package first



## Insights

#### There was a dramatic increase in sales during the first two months of the year. I would conduct further investigation into this to understand what happened so the company can potentially replicate it. (Was it a marketing campaign, for example?)

#### Electronics are by far the most popular product category among customers (46.9%), followed by Home & Garden (14.8%). This led me to question whether this strong performance was driven by the surge at the beginning of the year or whether Electronics consistently perform well throughout the year.

#### Electronics outperform all other categories even when the first two months are excluded. Based on this, I would suggest evaluating the cost of maintaining the other product categories and potentially specializing the store in Electronics. Books, for instance, accounted for only 1.4% of total revenue in 2023.